In [1]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import backend as K
from keras.optimizers import RMSprop
import keras

Using TensorFlow backend.


In [2]:
# allocates GPU memory based on runtime allocations. doesn't releases memory because of potential memory fragmentation
# without this option, getting images will not work
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
# Import the dataset
from keras.datasets import mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [6]:
# 60,000 items with dimensions 28x28x1
img_rows, img_cols = 28, 28
# digits 0-9
num_classes=10

In [7]:
# example of how it looks like: row, col, color
test = x_train[47].reshape(28, 28, 1)
array_to_img(test)

In [8]:
# Change shape 
# images are 28*28 pixels, so reshaping to 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, img_rows*img_cols)
x_test = x_test.reshape(10000, img_rows*img_cols)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [9]:
# simple sequential model
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(img_rows*img_cols,)))
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('basic_model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 35us/step - loss: 0.3929 - acc: 0.8933 - val_loss: 0.2287 - val_acc: 0.9342
Epoch 2/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.2045 - acc: 0.9417 - val_loss: 0.1714 - val_acc: 0.9494
Epoch 3/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.1560 - acc: 0.9556 - val_loss: 0.1418 - val_acc: 0.9570
Epoch 4/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.1264 - acc: 0.9630 - val_loss: 0.1316 - val_acc: 0.9613
Epoch 5/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.1066 - acc: 0.9692 - val_loss: 0.1092 - val_acc: 0.9681
Epoch 6/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.0920 - acc: 0.9728 - val_loss: 0.1051 - val_acc: 0.9694
Epoch 7/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.0822 - acc: 0.9757 - val_loss: 0.1040 - val_acc

In [11]:
# simple sequential model with dropout
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(img_rows*img_cols,)))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('basic_drop_model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 36us/step - loss: 0.4200 - acc: 0.8832 - val_loss: 0.2240 - val_acc: 0.9339
Epoch 2/10
60000/60000 [==============================] - 2s 35us/step - loss: 0.2175 - acc: 0.9366 - val_loss: 0.1650 - val_acc: 0.9508
Epoch 3/10
60000/60000 [==============================] - 2s 35us/step - loss: 0.1685 - acc: 0.9501 - val_loss: 0.1363 - val_acc: 0.9603
Epoch 4/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.1418 - acc: 0.9589 - val_loss: 0.1221 - val_acc: 0.9653
Epoch 5/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.1249 - acc: 0.9637 - val_loss: 0.1091 - val_acc: 0.9675
Epoch 6/10
60000/60000 [==============================] - 2s 33us/step - loss: 0.1105 - acc: 0.9666 - val_loss: 0.1018 - val_acc: 0.9705
Epoch 7/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.1013 - acc: 0.9694 - val_loss: 0.0963 - val_acc

In [13]:
score = model.evaluate(x_test[:1000], y_test[:1000])
print(score[1])

1000/1000 [==============================] - 0s 43us/step
0.972


In [14]:
prediction = model.predict(x_test).argmax(axis=-1)
actual = y_test.argmax(axis=-1)

In [15]:
import pandas as pd

num_cross = pd.crosstab(prediction, actual, rownames=['Prediction'], colnames=['Actual'])
num_cross

Actual,0,1,2,3,4,5,6,7,8,9
Prediction,,,,,,,,,,
0,971,0,4,1,1,2,6,2,4,3
1,0,1123,1,0,0,0,3,8,2,5
2,0,4,1006,6,3,0,1,16,5,0
3,1,0,2,983,0,8,1,4,3,9
4,0,0,3,0,950,1,5,0,3,9
5,1,1,1,4,1,863,2,0,5,3
6,4,4,4,0,10,9,938,0,2,0
7,1,0,4,5,2,1,0,986,5,5
8,2,3,7,11,2,7,2,2,944,5


In [16]:
# 2 layer model
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(img_rows*img_cols,)))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('2layer_model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.4334 - 

In [17]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# CNN model
model = Sequential()
# First convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('cnn_model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 142us/step - loss: 0.2695 - acc: 0.9168 - val_loss: 0.0554 - val_acc: 0.9806
Epoch 2/10
60000/60000 [==============================] - 7s 115us/step - loss: 0.0886 - acc: 0.9735 - val_loss: 0.0420 - val_acc: 0.9861
Epoch 3/10
60000/60000 [==============================] - 7s 116us/step - loss: 0.0650 - acc: 0.9805 - val_loss: 0.0335 - val_acc: 0.9885
Epoch 4/10
60000/60000 [==============================] - 7s 115us/step - loss: 0.0557 - acc: 0.9831 - val_loss: 0.0318 - val_acc: 0.9891
Epoch 5/10
60000/60000 [==============================] - 7s 117us/step - loss: 0.0458 - acc: 0.9861 - val_loss: 0.0295 - val_acc: 0.9899
Epoch 6/10
60000/60000 [==============================] - 7s 117us/step - loss: 0.0418 - acc: 0.9878 - val_loss: 0.0312 - val_acc: 0.9907
Epoch 7/10
60000/60000 [==============================] - 7s 117us/step - loss: 0.0359 - acc: 0.9885 - val_loss: 0.0269 - 

In [18]:
# CNN model with batch normalization
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('cnn_batch_normalization_model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 22s 361us/step - loss: 0.1897 - acc: 0.9476 - val_loss: 0.0479 - val_acc: 0.9843
Epoch 2/10
60000/60000 [==============================] - 20s 332us/step - loss: 0.0744 - acc: 0.9789 - val_loss: 0.0352 - val_acc: 0.9890
Epoch 3/10
60000/60000 [==============================] - 20s 331us/step - loss: 0.0550 - acc: 0.9842 - val_loss: 0.0304 - val_acc: 0.9898
Epoch 4/10
60000/60000 [==============================] - 20s 331us/step - loss: 0.0445 - acc: 0.9871 - val_loss: 0.0354 - val_acc: 0.9881
Epoch 5/10
60000/60000 [==============================] - 20s 331us/step - loss: 0.0376 - acc: 0.9891 - val_loss: 0.0288 - val_acc: 0.9897
Epoch 6/10
60000/60000 [==============================] - 20s 330us/step - loss: 0.0341 - acc: 0.9900 - val_loss: 0.0271 - val_acc: 0.9915
Epoch 7/10
60000/60000 [==============================] - 20s 331us/step - loss: 0.0287 - acc: 0.9910 - val_loss: 0.

In [19]:
prediction = model.predict(x_test).argmax(axis=-1)
actual = y_test.argmax(axis=-1)

num_cross = pd.crosstab(prediction, actual, rownames=['Prediction'], colnames=['Actual'])
num_cross

Actual,0,1,2,3,4,5,6,7,8,9
Prediction,,,,,,,,,,
0,978,0,1,0,0,2,3,0,3,0
1,0,1133,1,0,0,0,2,2,1,1
2,0,0,1025,1,0,0,0,4,1,0
3,0,0,0,1001,0,3,1,1,0,1
4,0,0,1,0,978,0,1,0,0,3
5,0,0,0,3,0,886,1,0,0,1
6,1,1,0,0,2,1,949,0,0,0
7,1,0,4,2,0,0,0,1018,0,4
8,0,1,0,3,0,0,1,1,968,3
